In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import json
import dateparser
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
docxFileName = "quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [ ]:
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'

mainstation = ""
initial_letter = ""
lowerstation = ""
dText = dict()
counter = 0
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    if is_upper(para) == 'station':
        mainxpath = para.xpath(mainstation_xpath, namespaces=ns)
        # Filter out wrongly formatted substations:
        if mainxpath and len(mainxpath[0].text) > 1 and not mainxpath[0].text.startswith(initial_letter + " "):
            print(mainxpath[0].text)
            counter += 1
            mainstation = mainxpath[0].text
            initial_letter = mainstation[0]
    description = description.lstrip('\x01').strip()
    if description:
        if (counter, mainstation) in dText:
            dText[(counter, mainstation)].append(description)# += "\n" + description
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[(counter, mainstation)] = [description]

In [ ]:
dStations = collections.OrderedDict(dText)

In [ ]:
indices = []
stations = []
descriptions = []
for k in dStations:
    indices.append(k[0])
    stations.append(k[1])
    descriptions.append(dStations[k])

In [ ]:
stationdf = pd.DataFrame(columns=["Index", "Station", "Description"])
stationdf["Index"] = indices
stationdf["Station"] = stations
stationdf["Description"] = descriptions
stationdf = stationdf.set_index("Index")

In [ ]:
stationdf.head()

In [ ]:
stationdf.to_pickle('quick_processed.pkl')

In [ ]:
stationdf_sample = stationdf.sample(n=100, random_state=42)

# Get 100 random entries:
for i, row in stationdf_sample.iterrows():
    with open("to_annotate/" + str(i) + ".txt", "w") as fw:
        fw.write("[[[STATION: " + row["Station"] + "]]]")
        for d in row["Description"]:
            fw.write("\n" + d)

# Get 25 long entries:
stationdf_sample = stationdf.sample(frac=1, random_state=42)
counter = 1
for i, row in stationdf_sample.iterrows():
    if len(row['Description']) >= 10:
        with open("to_annotate/" + str(i) + ".txt", "w") as fw:
            fw.write("[[[STATION: " + row["Station"] + "]]]")
            for d in row["Description"]:
                fw.write("\n" + d)
        if counter >= 25:
            break
        counter += 1